# Strategic Patient Risk Stratification & Readmission Predictive Modeling
## Vitality Health Network (VHN)

**Course:** ITS 2122: Python for Data Science & AI (Semester 3 – 2025)  
**Dataset:** Diabetes 130-US Hospitals (1999–2008)  
**Objective:** Analyze historical hospital data to identify drivers of 30-day readmissions and build a risk stratification system

---

## Table of Contents
1. [Phase 1: Data Ingestion & Clinical Sanitation](#phase1)
2. [Phase 2: Data Enrichment via Web Scraping](#phase2)
3. [Phase 3: Exploratory Data Analysis](#phase3)
4. [Phase 4: Feature Engineering - Vitality Complexity Index](#phase4)

---

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from utils import (
    calculate_vci_score,
    categorize_vci_risk,
    scrape_icd9_description,
    audit_data_quality,
    print_audit_summary,
    plot_readmission_by_category,
    plot_readmission_rate_by_category,
    create_correlation_heatmap
)

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

print("✓ Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

✓ Libraries imported successfully
Pandas version: 2.3.3
NumPy version: 2.3.5


<a id='phase1'></a>
# Phase 1: Data Ingestion & Clinical Sanitation

In this phase, we perform professional healthcare data cleaning:
- Load and audit the dataset
- Handle missing values and data quality issues
- Remove deceased patients
- Convert data types appropriately
- Remove duplicates
- Document all cleaning decisions with clinical rationale

## 1.1 Load Datasets

In [2]:
# Load main dataset
df = pd.read_csv('data_files/diabetic_data.csv')
print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Load ID mappings
id_mapping = pd.read_csv('data_files/IDs_mapping.csv')
print(f"\nID mapping loaded: {id_mapping.shape[0]} mappings")
print("\nFirst few rows of the dataset:")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data_files/diabetic_data.csv'

## 1.2 Initial Data Audit

In [1]:
# Display dataset information
print("=" * 70)
print("DATASET STRUCTURE")
print("=" * 70)
df.info()

DATASET STRUCTURE


NameError: name 'df' is not defined

In [ ]:
# Statistical summary
print("\n" + "=" * 70)
print("STATISTICAL SUMMARY - NUMERICAL FEATURES")
print("=" * 70)
df.describe()


STATISTICAL SUMMARY - NUMERICAL FEATURES


NameError: name 'df' is not defined

In [ ]:
# Check for '?' values (common placeholder in healthcare data)
print("\n" + "=" * 70)
print("CHECKING FOR '?' PLACEHOLDER VALUES")
print("=" * 70)

question_mark_counts = {}
for col in df.columns:
    if df[col].dtype == 'object':
        count = (df[col] == '?').sum()
        if count > 0:
            question_mark_counts[col] = count

if question_mark_counts:
    qm_df = pd.DataFrame.from_dict(question_mark_counts, orient='index', columns=['Count'])
    qm_df['Percentage'] = (qm_df['Count'] / len(df) * 100).round(2)
    qm_df = qm_df.sort_values('Count', ascending=False)
    print(qm_df)
else:
    print("No '?' values found")


CHECKING FOR '?' PLACEHOLDER VALUES


NameError: name 'df' is not defined

## 1.3 Convert '?' to NaN

**Clinical Rationale:** The '?' symbol is used as a placeholder for missing data in many healthcare datasets. Converting these to NaN allows proper handling with pandas missing data methods and prevents these values from being treated as valid categories.

In [ ]:
# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)
print("✓ Converted all '?' values to NaN")

# Verify conversion
print(f"\nTotal NaN values in dataset: {df.isna().sum().sum():,}")

## 1.4 Comprehensive Data Quality Audit

In [ ]:
# Perform audit using utility function
audit_results = audit_data_quality(df)
print_audit_summary(audit_results)

## 1.5 Handle High Missingness Columns

**Clinical Rationale:** Columns with >90% missing data provide minimal analytical value and can introduce bias. Common examples in healthcare data:
- **weight**: Often not recorded consistently across facilities
- **payer_code**: May not be captured in all systems
- **medical_specialty**: Frequently missing in administrative data

We document these as **data quality limitations** rather than attempting imputation, which would be clinically inappropriate.

In [ ]:
# Identify columns with >90% missingness
high_missing_threshold = 0.90
missing_pct = df.isna().sum() / len(df)
high_missing_cols = missing_pct[missing_pct > high_missing_threshold].index.tolist()

print(f"Columns with >{high_missing_threshold*100}% missing data:")
for col in high_missing_cols:
    pct = missing_pct[col] * 100
    print(f"  • {col}: {pct:.2f}% missing")

# Drop these columns
if high_missing_cols:
    df.drop(columns=high_missing_cols, inplace=True)
    print(f"\n✓ Dropped {len(high_missing_cols)} columns with extreme missingness")
    print(f"New dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
else:
    print("\nNo columns exceed the 90% missingness threshold")

## 1.6 Remove Deceased Patients

**Clinical Rationale:** Patients who expired (died) during hospitalization cannot be readmitted and should be excluded from readmission analysis. Including them would:
1. Artificially inflate the "no readmission" group
2. Skew risk models toward end-of-life care patterns
3. Violate the clinical definition of readmission risk

We use the discharge_disposition_id mapping to identify deceased patients.

In [ ]:
# Display discharge disposition mappings
print("Discharge Disposition ID Mappings:")
print(id_mapping[id_mapping['Table'] == 'discharge_disposition_id'])

# Identify deceased patient codes
# Typically codes 11, 13, 14, 19, 20, 21 indicate expired/hospice/deceased
deceased_codes = [11, 13, 14, 19, 20, 21]

print(f"\nDeceased/Expired discharge codes: {deceased_codes}")
print(f"Patients before removal: {len(df):,}")

# Count deceased patients
deceased_count = df[df['discharge_disposition_id'].isin(deceased_codes)].shape[0]
print(f"Deceased patients identified: {deceased_count:,} ({deceased_count/len(df)*100:.2f}%)")

# Remove deceased patients
df = df[~df['discharge_disposition_id'].isin(deceased_codes)].copy()
print(f"Patients after removal: {len(df):,}")
print(f"\n✓ Removed {deceased_count:,} deceased patients from analysis")

## 1.7 Convert to Appropriate Data Types

**Clinical Rationale:** Proper data typing improves:
- Memory efficiency
- Analysis accuracy
- Categorical analysis capabilities

In [ ]:
# Define categorical columns
categorical_cols = [
    'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
    'admission_source_id', 'max_glu_serum', 'A1Cresult',
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
    'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
    'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
    'insulin', 'glyburide-metformin', 'glipizide-metformin',
    'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone',
    'change', 'diabetesMed', 'readmitted'
]

# Convert to category dtype
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("✓ Converted categorical columns to 'category' dtype")
print(f"\nMemory usage after conversion: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 1.8 Remove Duplicate Records

**Clinical Rationale:** Duplicate records can occur due to:
- Data entry errors
- System integration issues
- Multiple submissions

Duplicates must be removed to ensure accurate statistical analysis and prevent bias in predictive models.

In [ ]:
# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows found: {duplicate_count:,}")

if duplicate_count > 0:
    df.drop_duplicates(inplace=True)
    print(f"✓ Removed {duplicate_count:,} duplicate rows")
else:
    print("✓ No duplicate rows found")

print(f"\nFinal dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

## 1.9 Phase 1 Summary

**Data Cleaning Decisions Summary:**

In [ ]:
print("=" * 70)
print("PHASE 1: DATA SANITATION COMPLETE")
print("=" * 70)
print("\nCleaning Actions Performed:")
print("  1. ✓ Converted '?' placeholders to NaN")
print(f"  2. ✓ Dropped {len(high_missing_cols) if high_missing_cols else 0} columns with >90% missingness")
print(f"  3. ✓ Removed {deceased_count:,} deceased patients")
print("  4. ✓ Converted categorical columns to appropriate dtype")
print(f"  5. ✓ Removed {duplicate_count:,} duplicate records")
print(f"\nFinal Clean Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
print("=" * 70)

<a id='phase2'></a>
# Phase 2: Data Enrichment via Web Scraping

In this phase, we enhance diagnosis data with medical meaning by:
- Identifying top 20 most frequent ICD-9 codes
- Building ethical web scraping script
- Scraping ICD-9 descriptions from public sources
- Adding Primary_Diagnosis_Desc column

## 2.1 Analyze Primary Diagnosis Codes

In [ ]:
# Analyze diag_1 (primary diagnosis)
print("=" * 70)
print("PRIMARY DIAGNOSIS (diag_1) ANALYSIS")
print("=" * 70)

# Get value counts
diag1_counts = df['diag_1'].value_counts()
print(f"\nTotal unique ICD-9 codes in diag_1: {len(diag1_counts):,}")
print(f"Missing values: {df['diag_1'].isna().sum():,}")

# Top 20 most frequent codes
top_20_codes = diag1_counts.head(20)
print("\nTop 20 Most Frequent ICD-9 Codes:")
print(top_20_codes)

# Calculate coverage
coverage = (top_20_codes.sum() / len(df)) * 100
print(f"\nTop 20 codes cover {coverage:.2f}% of all patients")

In [ ]:
# Visualize top 20 codes
plt.figure(figsize=(12, 6))
top_20_codes.plot(kind='bar', color='steelblue', alpha=0.8)
plt.title('Top 20 Most Frequent Primary Diagnosis Codes (ICD-9)', fontsize=14, fontweight='bold')
plt.xlabel('ICD-9 Code', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 2.2 Web Scraping ICD-9 Descriptions

**Ethical Scraping Practices:**
- Using public medical coding reference (ICD9Data.com)
- Implementing delays between requests (time.sleep)
- Setting appropriate User-Agent headers
- Handling errors gracefully
- Respecting rate limits

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# Get list of top 20 codes
top_20_list = top_20_codes.index.tolist()

print("Starting web scraping for top 20 ICD-9 codes...")
print("This may take a few moments due to ethical delay between requests.\n")

# Dictionary to store code descriptions
icd9_descriptions = {}

# Scrape descriptions
for i, code in enumerate(top_20_list, 1):
    print(f"[{i}/20] Scraping ICD-9 code: {code}...", end=' ')
    
    description = scrape_icd9_description(code, delay=0.5)
    icd9_descriptions[str(code)] = description
    
    print(f"✓ {description}")

print("\n✓ Web scraping complete!")

In [ ]:
# Display scraped descriptions
print("=" * 70)
print("SCRAPED ICD-9 DESCRIPTIONS")
print("=" * 70)

desc_df = pd.DataFrame.from_dict(icd9_descriptions, orient='index', columns=['Description'])
desc_df.index.name = 'ICD-9 Code'
desc_df['Frequency'] = desc_df.index.map(lambda x: diag1_counts.get(x, 0))
print(desc_df)

## 2.3 Add Primary_Diagnosis_Desc Column

In [ ]:
# Create mapping function
def map_diagnosis_description(code):
    """Map ICD-9 code to description, label non-top-20 as 'Other'"""
    if pd.isna(code):
        return 'Missing'
    code_str = str(code)
    return icd9_descriptions.get(code_str, 'Other')

# Apply mapping
df['Primary_Diagnosis_Desc'] = df['diag_1'].apply(map_diagnosis_description)

print("✓ Added 'Primary_Diagnosis_Desc' column")
print(f"\nValue distribution:")
print(df['Primary_Diagnosis_Desc'].value_counts())

## 2.4 Phase 2 Summary

In [ ]:
print("=" * 70)
print("PHASE 2: DATA ENRICHMENT COMPLETE")
print("=" * 70)
print("\nEnrichment Actions Performed:")
print(f"  1. ✓ Identified top 20 ICD-9 codes (covering {coverage:.2f}% of patients)")
print(f"  2. ✓ Scraped {len(icd9_descriptions)} ICD-9 descriptions from ICD9Data.com")
print("  3. ✓ Implemented ethical scraping practices (delays, headers, error handling)")
print("  4. ✓ Added 'Primary_Diagnosis_Desc' column")
print("  5. ✓ Labeled non-top-20 diagnoses as 'Other'")
print("=" * 70)

<a id='phase3'></a>
# Phase 3: Exploratory Data Analysis (EDA)

In this phase, we discover patterns and insights driving readmissions:
- Analyze class imbalance
- Visualize demographic patterns
- Compare medication usage
- Analyze operational metrics
- Create correlation analyses

## 3.1 Class Imbalance Analysis

In [ ]:
# Analyze readmission distribution
print("=" * 70)
print("READMISSION CLASS DISTRIBUTION")
print("=" * 70)

readmit_counts = df['readmitted'].value_counts()
readmit_pct = df['readmitted'].value_counts(normalize=True) * 100

readmit_summary = pd.DataFrame({
    'Count': readmit_counts,
    'Percentage': readmit_pct.round(2)
})

print(readmit_summary)

# Calculate 30-day readmission rate
readmit_30_rate = (df['readmitted'] == '<30').sum() / len(df) * 100
print(f"\n30-Day Readmission Rate: {readmit_30_rate:.2f}%")
print(f"This is {'ABOVE' if readmit_30_rate > 15 else 'BELOW'} the typical 15-20% benchmark")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
readmit_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c', '#3498db'], alpha=0.8)
axes[0].set_title('Readmission Status Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Readmission Status', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Pie chart
axes[1].pie(readmit_counts, labels=readmit_counts.index, autopct='%1.1f%%',
            colors=['#2ecc71', '#e74c3c', '#3498db'], startangle=90)
axes[1].set_title('Readmission Status Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Interpretation: The dataset shows class imbalance, with <30 day readmissions")
print("being the minority class. This is clinically realistic and will inform our")
print("risk stratification approach.")

## 3.2 Demographic Analysis

### 3.2.1 Age Analysis

In [ ]:
# Readmission by age group
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
sns.countplot(data=df, x='age', hue='readmitted', ax=axes[0], palette='Set2')
axes[0].set_title('Readmission Status by Age Group', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Age Group', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Readmitted', loc='upper left')

# Readmission rate by age
age_readmit_rate = df.groupby('age')['readmitted'].apply(
    lambda x: (x == '<30').sum() / len(x) * 100
).sort_index()

age_readmit_rate.plot(kind='bar', ax=axes[1], color='coral', alpha=0.8)
axes[1].set_title('30-Day Readmission Rate by Age Group', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Age Group', fontsize=12)
axes[1].set_ylabel('Readmission Rate (%)', fontsize=12)
axes[1].axhline(y=readmit_30_rate, color='red', linestyle='--', label=f'Overall: {readmit_30_rate:.2f}%')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Clinical Interpretation:")
print(f"Highest readmission rate: {age_readmit_rate.idxmax()} ({age_readmit_rate.max():.2f}%)")
print(f"Lowest readmission rate: {age_readmit_rate.idxmin()} ({age_readmit_rate.min():.2f}%)")
print("Age appears to be a significant factor in readmission risk.")

### 3.2.2 Gender Analysis

In [ ]:
# Readmission by gender
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sns.countplot(data=df, x='gender', hue='readmitted', ax=axes[0], palette='Set2')
axes[0].set_title('Readmission Status by Gender', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Gender', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].legend(title='Readmitted')

# Readmission rate by gender
gender_readmit_rate = df.groupby('gender')['readmitted'].apply(
    lambda x: (x == '<30').sum() / len(x) * 100
)

gender_readmit_rate.plot(kind='bar', ax=axes[1], color='skyblue', alpha=0.8)
axes[1].set_title('30-Day Readmission Rate by Gender', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Gender', fontsize=12)
axes[1].set_ylabel('Readmission Rate (%)', fontsize=12)
axes[1].axhline(y=readmit_30_rate, color='red', linestyle='--', label=f'Overall: {readmit_30_rate:.2f}%')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Clinical Interpretation:")
print(gender_readmit_rate)
print(f"Gender difference in readmission: {abs(gender_readmit_rate.iloc[0] - gender_readmit_rate.iloc[1]):.2f}%")

### 3.2.3 Race Analysis

In [ ]:
# Readmission by race
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
race_order = df['race'].value_counts().index
sns.countplot(data=df, x='race', hue='readmitted', ax=axes[0], palette='Set2', order=race_order)
axes[0].set_title('Readmission Status by Race', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Race', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Readmitted', loc='upper right')

# Readmission rate by race
race_readmit_rate = df.groupby('race')['readmitted'].apply(
    lambda x: (x == '<30').sum() / len(x) * 100
).sort_values(ascending=False)

race_readmit_rate.plot(kind='bar', ax=axes[1], color='lightcoral', alpha=0.8)
axes[1].set_title('30-Day Readmission Rate by Race', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Race', fontsize=12)
axes[1].set_ylabel('Readmission Rate (%)', fontsize=12)
axes[1].axhline(y=readmit_30_rate, color='red', linestyle='--', label=f'Overall: {readmit_30_rate:.2f}%')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Clinical Interpretation:")
print(race_readmit_rate)
print("\nRacial disparities in readmission rates may reflect social determinants")
print("of health, access to care, and systemic healthcare inequities.")

## 3.3 Medication Usage Analysis

In [ ]:
# Create medication categories
def categorize_medication(row):
    """Categorize patients by medication usage"""
    insulin = row.get('insulin', 'No')
    diabetesMed = row.get('diabetesMed', 'No')
    
    if insulin in ['Down', 'Steady', 'Up']:
        return 'Insulin User'
    elif diabetesMed == 'Yes':
        return 'Oral Medication'
    else:
        return 'No Medication'

df['Medication_Category'] = df.apply(categorize_medication, axis=1)

print("Medication Category Distribution:")
print(df['Medication_Category'].value_counts())

In [ ]:
# Analyze readmission by medication category
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
med_order = df['Medication_Category'].value_counts().index
sns.countplot(data=df, x='Medication_Category', hue='readmitted', ax=axes[0], 
              palette='Set2', order=med_order)
axes[0].set_title('Readmission Status by Medication Category', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Medication Category', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].tick_params(axis='x', rotation=15)
axes[0].legend(title='Readmitted')

# Readmission rate
med_readmit_rate = df.groupby('Medication_Category')['readmitted'].apply(
    lambda x: (x == '<30').sum() / len(x) * 100
).sort_values(ascending=False)

med_readmit_rate.plot(kind='bar', ax=axes[1], color='mediumseagreen', alpha=0.8)
axes[1].set_title('30-Day Readmission Rate by Medication Category', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Medication Category', fontsize=12)
axes[1].set_ylabel('Readmission Rate (%)', fontsize=12)
axes[1].axhline(y=readmit_30_rate, color='red', linestyle='--', label=f'Overall: {readmit_30_rate:.2f}%')
axes[1].tick_params(axis='x', rotation=15)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Clinical Interpretation:")
print(med_readmit_rate)
print("\nMedication management appears to influence readmission risk.")
print("Insulin users may represent more severe diabetes cases.")

## 3.4 Operational Metrics Analysis

### 3.4.1 Time in Hospital

In [ ]:
# Analyze time in hospital by readmission status
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot
sns.boxplot(data=df, x='readmitted', y='time_in_hospital', ax=axes[0], palette='Set2')
axes[0].set_title('Time in Hospital by Readmission Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Readmission Status', fontsize=12)
axes[0].set_ylabel('Days in Hospital', fontsize=12)

# Violin plot
sns.violinplot(data=df, x='readmitted', y='time_in_hospital', ax=axes[1], palette='Set2')
axes[1].set_title('Distribution of Hospital Stay Length', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Readmission Status', fontsize=12)
axes[1].set_ylabel('Days in Hospital', fontsize=12)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n📊 Time in Hospital Statistics by Readmission Status:")
print(df.groupby('readmitted')['time_in_hospital'].describe())

print("\nClinical Interpretation: Longer hospital stays may indicate higher")
print("complexity and could be associated with readmission risk.")

### 3.4.2 Number of Lab Procedures

In [ ]:
# Analyze lab procedures
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot
sns.boxplot(data=df, x='readmitted', y='num_lab_procedures', ax=axes[0], palette='Set2')
axes[0].set_title('Number of Lab Procedures by Readmission Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Readmission Status', fontsize=12)
axes[0].set_ylabel('Number of Lab Procedures', fontsize=12)

# Violin plot
sns.violinplot(data=df, x='readmitted', y='num_lab_procedures', ax=axes[1], palette='Set2')
axes[1].set_title('Distribution of Lab Procedures', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Readmission Status', fontsize=12)
axes[1].set_ylabel('Number of Lab Procedures', fontsize=12)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n📊 Lab Procedures Statistics by Readmission Status:")
print(df.groupby('readmitted')['num_lab_procedures'].describe())

print("\nClinical Interpretation: More lab procedures may indicate higher acuity")
print("and more complex medical conditions.")

### 3.4.3 Number of Medications

In [ ]:
# Analyze medications
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot
sns.boxplot(data=df, x='readmitted', y='num_medications', ax=axes[0], palette='Set2')
axes[0].set_title('Number of Medications by Readmission Status', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Readmission Status', fontsize=12)
axes[0].set_ylabel('Number of Medications', fontsize=12)

# Violin plot
sns.violinplot(data=df, x='readmitted', y='num_medications', ax=axes[1], palette='Set2')
axes[1].set_title('Distribution of Medications', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Readmission Status', fontsize=12)
axes[1].set_ylabel('Number of Medications', fontsize=12)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n📊 Medications Statistics by Readmission Status:")
print(df.groupby('readmitted')['num_medications'].describe())

print("\nClinical Interpretation: Higher medication counts suggest polypharmacy,")
print("which can increase complexity of care and potential for adverse events.")

## 3.5 Correlation Analysis

In [ ]:
# Select key numerical features for correlation analysis
numerical_features = [
    'time_in_hospital', 'num_lab_procedures', 'num_procedures',
    'num_medications', 'number_outpatient', 'number_emergency',
    'number_inpatient', 'number_diagnoses'
]

# Create correlation heatmap
corr_matrix = create_correlation_heatmap(df, columns=numerical_features, figsize=(12, 10))
plt.show()

print("\n📊 Key Correlations:")
print("\nStrongest positive correlations:")
# Get upper triangle of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
correlations = upper_tri.stack().sort_values(ascending=False)
print(correlations.head(5))

print("\nClinical Interpretation: Understanding feature correlations helps identify")
print("multicollinearity and reveals relationships between operational metrics.")

## 3.6 Phase 3 Summary

In [ ]:
print("=" * 70)
print("PHASE 3: EXPLORATORY DATA ANALYSIS COMPLETE")
print("=" * 70)
print("\nKey Findings:")
print(f"  1. ✓ 30-Day Readmission Rate: {readmit_30_rate:.2f}%")
print(f"  2. ✓ Class imbalance present (realistic for healthcare data)")
print("  3. ✓ Age shows variation in readmission risk")
print("  4. ✓ Medication category influences readmission patterns")
print("  5. ✓ Operational metrics (time, labs, meds) show associations")
print("  6. ✓ Correlations identified between clinical features")
print("\nThese insights will inform the VCI risk stratification model.")
print("=" * 70)

<a id='phase4'></a>
# Phase 4: Feature Engineering - Vitality Complexity Index (VCI)

In this phase, we build a patient risk scoring algorithm inspired by the LACE Index:
- **L**: Length of Stay (0-7 points)
- **A**: Acuity of Admission (0-3 points)
- **C**: Comorbidity Burden (0-5 points)
- **E**: Emergency Visits (0-5 points)

**Total VCI Score Range**: 0-20 points

## 4.1 Calculate VCI Scores

In [ ]:
# Calculate VCI score for each patient
print("Calculating Vitality Complexity Index (VCI) scores...")

df['VCI_Score'] = df.apply(calculate_vci_score, axis=1)

print("✓ VCI scores calculated")
print(f"\nVCI Score Statistics:")
print(df['VCI_Score'].describe())

# Display distribution
plt.figure(figsize=(12, 6))
plt.hist(df['VCI_Score'], bins=21, color='steelblue', alpha=0.7, edgecolor='black')
plt.title('Distribution of VCI Scores', fontsize=14, fontweight='bold')
plt.xlabel('VCI Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.axvline(df['VCI_Score'].mean(), color='red', linestyle='--', 
            label=f'Mean: {df["VCI_Score"].mean():.2f}')
plt.axvline(df['VCI_Score'].median(), color='green', linestyle='--',
            label=f'Median: {df["VCI_Score"].median():.2f}')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4.2 Categorize into Risk Groups

In [ ]:
# Categorize patients into risk groups
df['VCI_Risk_Category'] = df['VCI_Score'].apply(categorize_vci_risk)

print("Risk Category Distribution:")
risk_dist = df['VCI_Risk_Category'].value_counts()
print(risk_dist)
print(f"\nPercentage Distribution:")
print((risk_dist / len(df) * 100).round(2))

# Visualize risk categories
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
risk_order = ['Low', 'Medium', 'High']
risk_colors = ['#2ecc71', '#f39c12', '#e74c3c']
risk_dist_ordered = df['VCI_Risk_Category'].value_counts().reindex(risk_order)
risk_dist_ordered.plot(kind='bar', ax=axes[0], color=risk_colors, alpha=0.8)
axes[0].set_title('VCI Risk Category Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Risk Category', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Pie chart
axes[1].pie(risk_dist_ordered, labels=risk_order, autopct='%1.1f%%',
            colors=risk_colors, startangle=90)
axes[1].set_title('VCI Risk Category Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 4.3 Validate VCI Effectiveness

In [ ]:
# Calculate readmission rates by risk category
print("=" * 70)
print("VCI VALIDATION: 30-DAY READMISSION RATES BY RISK CATEGORY")
print("=" * 70)

risk_readmit_analysis = df.groupby('VCI_Risk_Category')['readmitted'].apply(
    lambda x: pd.Series({
        'Total_Patients': len(x),
        'Readmitted_30d': (x == '<30').sum(),
        'Readmission_Rate_%': (x == '<30').sum() / len(x) * 100
    })
).reindex(risk_order)

print(risk_readmit_analysis)

# Calculate risk ratio
low_rate = risk_readmit_analysis.loc['Low', 'Readmission_Rate_%']
high_rate = risk_readmit_analysis.loc['High', 'Readmission_Rate_%']
risk_ratio = high_rate / low_rate if low_rate > 0 else 0

print(f"\n📊 Key Validation Metrics:")
print(f"  • Low Risk Readmission Rate: {low_rate:.2f}%")
print(f"  • High Risk Readmission Rate: {high_rate:.2f}%")
print(f"  • Risk Ratio (High/Low): {risk_ratio:.2f}x")

if risk_ratio > 1.5:
    print("\n✅ VCI SUCCESSFULLY STRATIFIES RISK")
    print("High-risk patients have significantly higher readmission rates.")
else:
    print("\n⚠️ VCI shows moderate risk stratification")
    print("Further refinement may be needed.")

In [ ]:
# Visualize readmission rates by risk category
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Stacked bar chart
readmit_by_risk = pd.crosstab(df['VCI_Risk_Category'], df['readmitted'], normalize='index') * 100
readmit_by_risk = readmit_by_risk.reindex(risk_order)
readmit_by_risk.plot(kind='bar', stacked=True, ax=axes[0], 
                     color=['#2ecc71', '#e74c3c', '#3498db'], alpha=0.8)
axes[0].set_title('Readmission Status Distribution by Risk Category', fontsize=14, fontweight='bold')
axes[0].set_xlabel('VCI Risk Category', fontsize=12)
axes[0].set_ylabel('Percentage (%)', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].legend(title='Readmitted', bbox_to_anchor=(1.05, 1), loc='upper left')

# 30-day readmission rate comparison
readmit_30_by_risk = risk_readmit_analysis['Readmission_Rate_%']
bars = axes[1].bar(risk_order, readmit_30_by_risk, color=risk_colors, alpha=0.8)
axes[1].set_title('30-Day Readmission Rate by VCI Risk Category', fontsize=14, fontweight='bold')
axes[1].set_xlabel('VCI Risk Category', fontsize=12)
axes[1].set_ylabel('Readmission Rate (%)', fontsize=12)
axes[1].axhline(y=readmit_30_rate, color='red', linestyle='--', 
                label=f'Overall: {readmit_30_rate:.2f}%')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}%', ha='center', va='bottom', fontweight='bold')

axes[1].legend()

plt.tight_layout()
plt.show()

## 4.4 VCI Score vs Readmission Analysis

In [ ]:
# Analyze readmission rate across VCI score spectrum
vci_readmit_by_score = df.groupby('VCI_Score')['readmitted'].apply(
    lambda x: (x == '<30').sum() / len(x) * 100
).sort_index()

plt.figure(figsize=(14, 6))
plt.plot(vci_readmit_by_score.index, vci_readmit_by_score.values, 
         marker='o', linewidth=2, markersize=8, color='steelblue')
plt.title('30-Day Readmission Rate by VCI Score', fontsize=14, fontweight='bold')
plt.xlabel('VCI Score', fontsize=12)
plt.ylabel('Readmission Rate (%)', fontsize=12)
plt.axhline(y=readmit_30_rate, color='red', linestyle='--', alpha=0.7,
            label=f'Overall Average: {readmit_30_rate:.2f}%')
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print("\n📊 Clinical Interpretation:")
print("The trend line shows how readmission risk changes across the VCI spectrum.")
print("An upward trend validates the VCI as an effective risk stratification tool.")

## 4.5 Component Analysis

In [ ]:
# Analyze contribution of each VCI component
print("=" * 70)
print("VCI COMPONENT ANALYSIS")
print("=" * 70)

# Calculate individual component scores
def get_length_score(days):
    if days < 1: return 0
    elif 1 <= days <= 4: return 1
    elif 5 <= days <= 13: return 4
    else: return 7

df['L_Score'] = df['time_in_hospital'].apply(get_length_score)
df['A_Score'] = df['admission_type_id'].apply(lambda x: 3 if x in [1, 2] else 0)

# Analyze component distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Length of Stay
df['L_Score'].value_counts().sort_index().plot(kind='bar', ax=axes[0,0], 
                                                color='skyblue', alpha=0.8)
axes[0,0].set_title('L: Length of Stay Score Distribution', fontweight='bold')
axes[0,0].set_xlabel('Score')
axes[0,0].set_ylabel('Count')

# Acuity
df['A_Score'].value_counts().sort_index().plot(kind='bar', ax=axes[0,1],
                                                color='lightcoral', alpha=0.8)
axes[0,1].set_title('A: Acuity of Admission Score Distribution', fontweight='bold')
axes[0,1].set_xlabel('Score')
axes[0,1].set_ylabel('Count')

# Emergency Visits
df['number_emergency'].value_counts().sort_index().head(10).plot(kind='bar', ax=axes[1,0],
                                                                   color='lightgreen', alpha=0.8)
axes[1,0].set_title('E: Emergency Visits Distribution', fontweight='bold')
axes[1,0].set_xlabel('Number of Emergency Visits')
axes[1,0].set_ylabel('Count')

# Number of Diagnoses (proxy for Comorbidity)
df['number_diagnoses'].value_counts().sort_index().plot(kind='bar', ax=axes[1,1],
                                                         color='plum', alpha=0.8)
axes[1,1].set_title('C: Number of Diagnoses Distribution', fontweight='bold')
axes[1,1].set_xlabel('Number of Diagnoses')
axes[1,1].set_ylabel('Count')

plt.tight_layout()
plt.show()

print("\nEach component contributes to the overall VCI score, capturing different")
print("aspects of patient complexity and readmission risk.")

## 4.6 Phase 4 Summary & Final Results

In [ ]:
print("=" * 70)
print("PHASE 4: VITALITY COMPLEXITY INDEX (VCI) - COMPLETE")
print("=" * 70)
print("\nVCI Implementation Summary:")
print(f"  ✓ Calculated VCI scores for {len(df):,} patients")
print(f"  ✓ Score range: {df['VCI_Score'].min()}-{df['VCI_Score'].max()} (possible: 0-20)")
print(f"  ✓ Mean VCI: {df['VCI_Score'].mean():.2f}")
print(f"  ✓ Median VCI: {df['VCI_Score'].median():.2f}")
print("\nRisk Stratification Results:")
for risk in risk_order:
    count = risk_dist_ordered[risk]
    pct = (count / len(df)) * 100
    rate = risk_readmit_analysis.loc[risk, 'Readmission_Rate_%']
    print(f"  • {risk} Risk: {count:,} patients ({pct:.1f}%) - {rate:.2f}% readmission rate")

print(f"\n✅ VCI VALIDATION: Risk Ratio = {risk_ratio:.2f}x")
print("   High-risk patients are {:.1f}x more likely to be readmitted than low-risk.".format(risk_ratio))

print("\n" + "=" * 70)
print("PROJECT COMPLETE - ALL PHASES FINISHED")
print("=" * 70)
print("\nNext Steps:")
print("  1. Review Strategic Insight Report for business recommendations")
print("  2. Implement VCI in clinical workflow for discharge planning")
print("  3. Monitor VCI performance with prospective data")
print("  4. Refine risk thresholds based on operational capacity")
print("=" * 70)

## 4.7 Export Enhanced Dataset

In [ ]:
# Save enhanced dataset with VCI scores
output_file = 'VHN_Enhanced_Dataset_with_VCI.csv'
df.to_csv(output_file, index=False)
print(f"✓ Enhanced dataset saved to: {output_file}")
print(f"  Includes VCI_Score and VCI_Risk_Category columns")
print(f"  Total records: {len(df):,}")
print(f"  Total features: {len(df.columns)}")